Лаборатоная работа № 4

Шамсутдинов Рустам БВТ2201

In [1]:
import pygame
import random
import math

# Initialize pygame
pygame.init()

# Set up display
width, height = 800, 600
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Predator-Prey Game with Stars")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)  # Star color
RED = (255, 0, 0)
GRID_COLOR = (200, 200, 200)

# Speeds
CIRCLE_SPEED = 2
TRIANGLE_SPEED = 30  # Faster than the circle for better evasion

# Entity classes
class Circle:
    def __init__(self):
        self.x = random.randint(0, width)
        self.y = random.randint(0, height)
        self.color = BLACK  # Circle color
        self.radius = 10

    def move(self, target=None):
        if target:
            dx, dy = target[0] - self.x, target[1] - self.y
            dist_to_target = math.sqrt(dx**2 + dy**2)

            # Move towards target
            if dist_to_target > 0:
                self.x += int(CIRCLE_SPEED * dx / dist_to_target)
                self.y += int(CIRCLE_SPEED * dy / dist_to_target)
                return True  # Circle moved
        return False  # Circle did not move

    def draw(self, surface):
        pygame.draw.circle(surface, self.color, (self.x, self.y), self.radius)


class Triangle:
    def __init__(self):
        self.x = random.randint(0, width)
        self.y = random.randint(0, height)
        self.color = RED
        self.size = 15

    def move_away(self, threat):
        dx, dy = self.x - threat[0], self.y - threat[1]
        dist = math.sqrt(dx**2 + dy**2)
        if dist > 0 and dist < 100:  # Move away if the threat is close
            self.x += int(TRIANGLE_SPEED * dx / dist)
            self.y += int(TRIANGLE_SPEED * dy / dist)

        # Keep within screen boundaries
        self.x = max(0, min(self.x, width))
        self.y = max(0, min(self.y, height))

    def draw(self, surface):
        points = [(self.x, self.y - self.size),
                  (self.x - self.size, self.y + self.size),
                  (self.x + self.size, self.y + self.size)]
        pygame.draw.polygon(surface, self.color, points)


class Star:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.color = BLUE  # Star color
        self.size = 20

    def draw(self, surface):
        # Drawing star shape
        points = [
            (self.x, self.y - self.size),  # Top point
            (self.x + self.size // 2, self.y - self.size // 4),
            (self.x + self.size, self.y - self.size // 4),
            (self.x + self.size * 2 // 3, self.y + self.size // 4),
            (self.x + self.size * 3 // 4, self.y + self.size),
            (self.x, self.y + self.size // 2),
            (self.x - self.size * 3 // 4, self.y + self.size),
            (self.x - self.size * 2 // 3, self.y + self.size // 4),
            (self.x - self.size, self.y - self.size // 4),
            (self.x - self.size // 2, self.y - self.size // 4)
        ]
        pygame.draw.polygon(surface, self.color, points)


# Helper function to find the closest entity
def find_closest(circle, triangles):
    closest_triangle = None
    min_dist = float('inf')
    for triangle in triangles:
        dist = math.sqrt((triangle.x - circle.x) ** 2 + (triangle.y - circle.y) ** 2)
        if dist < min_dist:
            min_dist = dist
            closest_triangle = triangle
    return closest_triangle, min_dist


# Game variables
max_circles = 10  # Limit the number of circles in the game
creatures = {
    "circle": [Circle() for _ in range(3)],
    "triangle": [Triangle() for _ in range(5)],
}
stars = [Star(width // 2, height // 2), Star(width // 3, height // 3), Star(2 * width // 3, 2 * height // 3)]
game_over = False

# Main game loop
running = True
clock = pygame.time.Clock()
respawn_timer = 0  # Timer for triangle respawn

# Function to draw grid lines
def draw_grid(surface, grid_size=50):
    for x in range(0, width, grid_size):
        pygame.draw.line(surface, GRID_COLOR, (x, 0), (x, height))
    for y in range(0, height, grid_size):
        pygame.draw.line(surface, GRID_COLOR, (0, y), (width, y))

while running:
    window.fill(WHITE)

    # Draw grid for easier tracking
    draw_grid(window)

    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Update and draw creatures
    for circle in creatures['circle'][:]:  # Copy to safely remove within loop
        closest_triangle, min_dist = find_closest(circle, creatures['triangle'])
        
        # Check if circle is touching any star and remove if it is
        for star in stars:
            dist_to_star = math.sqrt((circle.x - star.x) ** 2 + (circle.y - star.y) ** 2)
            if dist_to_star < star.size:  # Circle is "melted" by the star
                creatures['circle'].remove(circle)
                break

        if not creatures['circle']:  # Game over if no circles left
            game_over = True
            break

        if closest_triangle and min_dist < 8:  # Eat triangle if close enough
            if closest_triangle in creatures['triangle']:  # Check if triangle is still present
                creatures['triangle'].remove(closest_triangle)
                # Spawn exactly two new circles if within the limit
                if len(creatures['circle']) < max_circles:
                    creatures['circle'].append(Circle())
                    creatures['circle'].append(Circle())
        else:
            # Move towards the closest triangle
            circle.move((closest_triangle.x, closest_triangle.y) if closest_triangle else None)
        circle.draw(window)

    if game_over:
        print("Game Over! All circles have melted or been eaten.")
        running = False
        break

    for triangle in creatures['triangle']:
        # Move triangle away from the closest circle if it’s within range
        closest_circle, dist_to_circle = find_closest(triangle, creatures['circle'])
        if closest_circle and dist_to_circle < 100:
            triangle.move_away((closest_circle.x, closest_circle.y))
        triangle.draw(window)

    # Draw stars
    for star in stars:
        star.draw(window)

    # Respawn triangles periodically
    respawn_timer += 1
    if respawn_timer > 120:  # Respawn every 2 seconds
        creatures['triangle'].append(Triangle())
        respawn_timer = 0

    # Update display
    pygame.display.flip()
    clock.tick(30)

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
